In [1]:
!pip install gradio
!pip install yfinance
!pip install plotly
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install re
!pip insall requests
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 937.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the 

In [3]:
from google.colab import userdata

In [4]:
import pandas as pd
import openai
import gradio as gr
import yfinance as yf
import json
import os
import matplotlib.pyplot as plt
import io
from PIL import Image
import plotly.graph_objs as go  # Import Plotly for interactive charts
import re
import datetime


In [5]:
# openai.api_key = os.getenv('OPENAI_API_KEY') or 'your-openai-api-key'  # Replace 'your-openai-api-key' with your actual key
openai.api_key = userdata.get('apopen')

## Test 1

In [2]:

# Set your OpenAI API key

# openai.api_key = os.getenv('OPENAI_API_KEY') or 'your-openai-api-key'  # Replace 'your-openai-api-key' with your actual key
openai.api_key = userdata.get('apopen')


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="N/A"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": safe_get(info, "marketCap"),
        "total_revenue": safe_get(info, "totalRevenue"),
        "gross_profit_margin": safe_get(info, "grossMargins"),
        "ebitda_margin": safe_get(info, "ebitdaMargins"),
        "operating_margin": safe_get(info, "operatingMargins"),
        "net_profit_margin": safe_get(info, "profitMargins"),
        "eps_diluted": safe_get(info, "trailingEps"),
        "pe_ratio": safe_get(info, "trailingPE"),
        "forward_pe_ratio": safe_get(info, "forwardPE")
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list

# Function to get stock performance and historical data
def get_stock_performance(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    history = stock.history(start=start_date, end=end_date)
    current_price = history['Close'][-1]

    performance = {
        "current_price": current_price,
        "history": history
    }
    return performance

# Function to plot the stock price history
def plot_stock_history(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.index, y=history['Close'], mode='lines', name='Close Price'))

    fig.update_layout(
        title='Stock Price History',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        hovermode='x unified'
    )
    return fig

# Function to generate the full stock report
def get_full_stock_report(ticker, start_date, end_date):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker, start_date, end_date)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Define the function schema for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Get a detailed stock report.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "Stock ticker symbol."
                },
                "start_date": {
                    "type": "string",
                    "description": "Start date for historical data in YYYY-MM-DD format."
                },
                "end_date": {
                    "type": "string",
                    "description": "End date for historical data in YYYY-MM-DD format."
                }
            },
            "required": ["ticker", "start_date", "end_date"]
        }
    }
]

# Function to handle the chat and function calling
def stock_chat(user_message, ticker, date_range):
    messages = [
        {
            "role": "system",
            "content": """You are a stock information bot. Provide detailed stock reports in the following format:

[Company Name] ([Ticker]) Overview
Company Name: ...
Ticker: ...
Exchange: ...
Industry: ...
CEO: ...
Year Founded: ...
Headquarters: ...

Description: ...

Financial Metrics & Fundamentals
Market Cap: ...
Total Revenues: ...
Gross Profit Margin: ...
EBITDA Margin: ...
Operating Margin: ...
Net Profit Margin: ...
EPS Diluted: ...
P/E Ratio: ...
Forward P/E Ratio: ...

Recent News
- News Item 1
- News Item 2
- ...

Stock Performance
Current Price: ...
52-Week Range: ...
YTD Total Return: ...
1Y Total Return: ...
5Y Total Return CAGR: ...
10Y Total Return CAGR: ...

Summary
[Provide a brief summary of the company's performance and outlook.]
"""
        },
        {"role": "user", "content": user_message}
    ]

    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=date_range)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use the latest model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=0
    )

    response_message = response["choices"][0]["message"]

    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        arguments = json.loads(response_message["function_call"]["arguments"])
        ticker = arguments.get("ticker")

        function_mapping = {
            "get_full_stock_report": get_full_stock_report
        }

        function_to_call = function_mapping.get(function_name)
        if function_to_call:
            function_response = function_to_call(ticker, start_date_str, end_date_str)

            # Format the report
            formatted_response = format_stock_report(function_response)

            # Add the assistant's message and function response to the messages
            messages.append(response_message)
            messages.append({
                "role": "function",
                "name": function_name,
                "content": formatted_response
            })

            # Get the assistant's final response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=0
            )

            return final_response['choices'][0]['message']['content'], plot_stock_history(function_response['stock_performance']['history'])
        else:
            return "Sorry, I can't handle that request at the moment."
    else:
        return response_message.get('content', "Sorry, I didn't understand your request.")

# Gradio Interface
iface = gr.Interface(
    fn=lambda user_message, ticker, date_range: stock_chat(user_message, ticker, date_range),
    inputs=[
        gr.Textbox(lines=1, placeholder="Enter stock ticker symbol (e.g., AAPL)", label="Stock Ticker"),
        gr.Slider(365, 1825, value=365, step=30, label="Date Range (in days)")
    ],
    outputs=[
        gr.Markdown(label="Stock Report"),
        gr.Plot(label="Stock Price Chart")
    ],
    title="Interactive Stock Analysis",
    description="Enter a stock ticker to see the company overview and stock price history with an adjustable date range.",
)

iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1002: UserWarning: Expected 3 arguments for function <function <lambda> at 0x7ad7acfc8310>, received 2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1006: UserWarning: Expected at least 3 arguments for function <function <lambda> at 0x7ad7acfc8310>, received 2.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4a474c16befbe1fa55.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
